### Probability Sharpe Ratio

In [1]:
import numpy as np
from FinancialMachineLearning.backtest.backtest_statistics import ProbSharpeRatio

In [2]:
import pandas as pd

import yfinance as yf

start_date = '2010-01-01'
end_date = '2024-01-01'

ibm = yf.download('IBM', start = start_date, end = end_date)
ibm_ret = np.log(ibm.Close / ibm.Close.shift(1)).dropna()
ibm_ret.head()

[*********************100%%**********************]  1 of 1 completed


Date
2010-01-05   -0.012154
2010-01-06   -0.006517
2010-01-07   -0.003468
2010-01-08    0.009985
2010-01-11   -0.010525
Name: Close, dtype: float64

In [3]:
apple = yf.download('AAPL', start = start_date, end = end_date)
apple_ret = np.log(apple.Close / apple.Close.shift(1)).dropna()
apple_ret.head()

[*********************100%%**********************]  1 of 1 completed


Date
2010-01-05    0.001727
2010-01-06   -0.016034
2010-01-07   -0.001850
2010-01-08    0.006626
2010-01-11   -0.008861
Name: Close, dtype: float64

In [4]:
google = yf.download('GOOGL', start = start_date, end = end_date)
google_ret = np.log(google.Close / google.Close.shift(1)).dropna()
google_ret.head()

[*********************100%%**********************]  1 of 1 completed


Date
2010-01-05   -0.004413
2010-01-06   -0.025532
2010-01-07   -0.023555
2010-01-08    0.013243
2010-01-11   -0.001513
Name: Close, dtype: float64

In [5]:
matrix = pd.concat([
    ibm_ret, apple_ret, google_ret
], axis = 1)
matrix.columns = ['ibm','apple','google']
matrix.head()

,ibm,apple,google
Date,,,
2010-01-05,-0.012154,0.001727,-0.004413
2010-01-06,-0.006517,-0.016034,-0.025532
2010-01-07,-0.003468,-0.001850,-0.023555
2010-01-08,0.009985,0.006626,0.013243
2010-01-11,-0.010525,-0.008861,-0.001513


In [6]:
matrix.values

array([[-0.01215355,  0.00172744, -0.00441338],
       [-0.0065172 , -0.01603419, -0.0255319 ],
       [-0.00346751, -0.00185042, -0.0235548 ],
       ...,
       [ 0.0015306 ,  0.00051782, -0.00815932],
       [ 0.00177252,  0.00222382, -0.00099786],
       [-0.0012221 , -0.00543889, -0.0038582 ]])

In [9]:
#1) Inputs (path to csv file with returns series)
#path = '../../Data/stock_prices.csv'
maxIter = 100 # Maximum number of iterations
delta = 0.1 # Delta Z (attempted gain per interation)

#2) Load data, set seed
#series = np.genfromtxt(path, delimiter = ',') # load as numpy array
seed = np.ones((matrix.values.shape[1], 1)) / matrix.values.shape[1] # initialize seed
bounds = [(0,1) for i in seed] # min and max boundary per weight

#3) Create class and solve
psrOpt = ProbSharpeRatio(matrix.values, seed, delta, maxIter, bounds)
psrOpt.optimize()

/Users/junghunlee/Desktop/Pycharm/FinancialMachineLearning/FinancialMachineLearning/backtest/backtest_statistics.py:120: RuntimeWarning: divide by zero encountered in scalar divide
  return self.get_dn_moments(mean, w, 2, 1, index) / (2 * sigma)
/Users/junghunlee/Desktop/Pycharm/FinancialMachineLearning/FinancialMachineLearning/backtest/backtest_statistics.py:134: RuntimeWarning: invalid value encountered in scalar multiply
  d1Skew -= 3 * sigma ** 2 * d1sigma * m3
/Users/junghunlee/Desktop/Pycharm/FinancialMachineLearning/FinancialMachineLearning/backtest/backtest_statistics.py:150: RuntimeWarning: invalid value encountered in scalar multiply
  d1Kurt -= 4 * sigma ** 3 * d1sigma * m4
/Users/junghunlee/Desktop/Pycharm/FinancialMachineLearning/FinancialMachineLearning/backtest/backtest_statistics.py:95: RuntimeWarning: invalid value encountered in scalar multiply
  d1meanSR = (d1Mu * stats[1] - d1Sigma * stats[0]) / stats[1] ** 2


In [10]:
print(f"Maximized Z-Value : {psrOpt.z}")
print(f"Number of iterations : {psrOpt.iter}")
print("PSR optimal portfolio")
psrOpt.w

Maximized Z-Value : 0
Number of iterations : 100
PSR optimal portfolio


array([[0.33333333],
       [0.33333333],
       [0.33333333]])